<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/index_relation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import ast

class Stock:
  def __init__(self, code):
    self.code = code
    self.headers = {'User-Agent' : 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36'}

  def get_price(self, start:int='20180501', interval='day'):
    url ='https://api.finance.naver.com/siseJson.naver?symbol=' + self.code + ' &requestType=1&startTime=' \
    + start + '&endTime=' + str(datetime.datetime.now().date().strftime("%Y%m%d")) +  '&timeframe=' + interval

    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content, "html.parser")
    b = bs_obj.get_text()
    for i in ['\n', '\t', "\\", ' ']:
      b = b.replace(i,'')

    data = np.array(ast.literal_eval(b)[1:])

    dic = {'Date':list(map(lambda x: datetime.datetime.strptime(str(x), '%Y%m%d'), data[:,0])), \
      'Open':np.array(data[:,1], int), 'High':np.array(data[:,2], int), 'Low':np.array(data[:,3], int),\
       'Close':np.array(data[:,4], int), 'Volume':np.array(data[:,5], int)}

    df = pd.DataFrame(data=dic)
    df = df.set_index('Date')
    return df
    
  def get_fundamental(self):
    url = 'https://finance.naver.com/item/main.nhn?code=' + self.code
    result = requests.get(url, headers = self.headers)
    bs_obj = BeautifulSoup(result.content, "html.parser")

    ths = bs_obj.find_all("th", {'scope':'col'})
    ths = [th.get_text() for th in ths][10:-22]
    dates = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),ths))
    dates = list(map(lambda x: x + '(Y)', dates[:4])) + list(map(lambda x: x + '(M)', dates[4:]))

    tr = bs_obj.find_all("tbody")
    tds = tr[2].find_all('td')
    tds = [td.get_text() for td in tds]
    elements = list(map(lambda x: x.translate(str.maketrans('','','\n\t, ')),tds))
    elements = list(map(lambda x: float(x) if x.replace('.','').isdigit() else np.nan, elements))

    temp_dict = {}
    cnt = 0
    index = ['sales', 'operating profit', 'net income', 'operating margin', 'net margin', 'roe', 'debt ratio', 'quick ratio', \
    'reserve ratio', 'eps', 'per', 'bps', 'pbr', 'dividend per share', 'dividend yield ratio', 'dividend payout ratio']

    for i in dates:
      temp_dict[i] = elements[cnt::10]
      cnt += 1

    df = pd.DataFrame.from_dict(temp_dict)
    df = df.set_index([pd.Index(index)])
    return df

In [5]:
stock = Stock('KOSPI')
stock.get_price('20000101', 'day')

,Open,High,Low,Close,Volume
Date,,,,,
2000-01-04,102833,106618,101659,105904,195898
2000-01-05,100687,102652,98405,98631,257696
2000-01-06,101395,101490,95350,96079,203523
2000-01-07,94917,97016,93084,94865,215664
2000-01-10,97967,99494,97482,98724,240175
...,...,...,...,...,...
2021-07-07,330359,330359,327766,328534,1147641
2021-07-08,328768,329571,325241,325268,1516034
2021-07-09,324552,324552,318880,321795,1308364


In [6]:
stock.get_price('20000101', 'week')

,Open,High,Low,Close,Volume
Date,,,,,
2000-01-07,102833,106618,93084,94865,872781
2000-01-14,97967,100587,93775,94803,1172282
2000-01-21,96267,99285,90626,92516,1141683
2000-01-28,91793,94431,87582,94167,1322198
2000-02-03,92483,95931,92292,95022,1183105
...,...,...,...,...,...
2021-06-18,324960,328196,323851,326793,9221163
2021-06-25,326418,331608,322556,330284,6743962
2021-07-02,330546,331169,327663,328178,6431414


In [7]:
stock.get_price('20000101', 'month')

,Open,High,Low,Close,Volume
Date,,,,,
2000-01-31,102833,106618,87582,94388,4798379
2000-02-29,95544,99171,81053,82838,5144645
2000-03-31,86478,92855,82030,86094,6536459
2000-04-28,86675,88041,68836,72539,4113517
2000-05-31,73176,77846,62514,73188,4940337
...,...,...,...,...,...
2021-03-31,302168,309650,292936,306142,27901970
2021-04-30,307377,322305,306968,314786,31066502
2021-05-31,314905,325590,310388,320392,18790308
